In [32]:
import requests

import json

#Librarías para todo o notebook
import pandas as pd
import pymysql

#import sqlalchemy
from sqlalchemy.engine import create_engine
from sqlalchemy import text

Establecemos la conexión a la base de datos

In [33]:
#Datos de conexión
db_host = "10.133.27.46"
db_port=9906
db_user = "nifi"
db_passwd="Nifi.Abc123"
db_name="nifi"

#Xerar a cadea de conexión en base aos parámetros anteriores
connectionString=f'mysql+pymysql://{db_user}:{db_passwd}@{db_host}:{db_port}/{db_name}'

Creamos el engine que vamos a utilizar y asociamos los datos del json extraído del api, para crear una sentencia SQL para insertar los datos en la tabla trono

In [34]:
engine = create_engine(connectionString)

enderezo = f"https://thronesapi.com/api/v2/Characters"
resposta = requests.get(enderezo)

resposta_json = resposta.json()

for personaje in resposta_json:
    id = personaje['id']
    firstName = personaje['firstName']
    lastName = personaje['lastName']
    fullName = personaje['fullName']
    title = personaje['title']
    family = personaje['family']
    image = personaje['image']
    imageUrl = personaje['imageUrl']

    cadenaSQL = f'INSERT INTO trono (id, firstName, lastName, fullName, title, family, image, imageUrl) VALUES ("{id}","{firstName}", "{lastName}", "{fullName}", "{title}", "{family}", "{image}", "{imageUrl}");'
    with engine.connect() as connection:
        result = connection.execute(text(cadenaSQL))
        connection.commit()



Seguimos usando el mismo engine pero esta vez los datos sacados de local dado que netflix bloquea las peticiones no autorizadas, y usando la tabla netflix

In [35]:
f = open("./json/netflix.json", "r")
content = f.read()
resposta_json2 = json.loads(content)

for pel in resposta_json2:
    videoID = pel['videoID']
    country = pel['country']
    title1 = pel['title1']
    title2 = pel['title2']
    startTime = pel['startTime']
    collection = pel['collection']
    if collection == "":
        collection = 0
    image = pel['image']
    genre = pel['genre']

    cadenaSQL = f'INSERT INTO netflix (videoID, country, title1, title2, startTime, collection, image, genre) VALUES ("{videoID}","{country}", "{title1}", "{title2}", "{startTime}", "{collection}", "{image}", "{genre}");'
    with engine.connect() as connection:
        result = connection.execute(text(cadenaSQL))
        connection.commit()


# Cerramos a conexión
engine.dispose()